- tested to be working 3/21/2019, given input file 'pdfs.tsv'

In [1]:
import os
from pathlib import Path

In [5]:
# Load Snorkel
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

n_docs = 500 if 'CI' in os.environ else 2591

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
from snorkel.parser import TSVDocPreprocessor

In [14]:
doc_preprocessor = TSVDocPreprocessor('pdfs.tsv', max_docs=n_docs)

In [16]:
corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)

Clearing existing...
Running UDF...





  0%|                                            | 0/2591 [00:00<?, ?it/s]


  0%|                                    | 2/2591 [00:02<48:13,  1.12s/it]


  0%|                                    | 3/2591 [00:04<58:25,  1.35s/it]


  0%|                                    | 4/2591 [00:04<51:54,  1.20s/it]


  0%|                                    | 5/2591 [00:05<46:29,  1.08s/it]


  0%|                                    | 6/2591 [00:07<51:45,  1.20s/it]


  0%|                                    | 7/2591 [00:08<53:10,  1.23s/it]


  0%|                                  | 8/2591 [00:10<1:05:23,  1.52s/it]


  0%|                                  | 9/2591 [00:12<1:11:45,  1.67s/it]


  0%|▏                                | 10/2591 [00:14<1:12:03,  1.68s/it]


  0%|▏                                | 11/2591 [00:16<1:17:52,  1.81s/it]


  0%|▏                                | 12/2591 [00:17<1:06:38,  1.55s/it]


  1%|▏                                | 13/2591 [00:18<1:00:18,  1.40s/it

Wall time: 1min 14s


In [17]:
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 39
Sentences: 14435


### Next steps to do: create matcher functions from dictionaries to match virus/host names in the text data

In [8]:
# Extract Candidates:
# First import classes to define candidates and their subclasses

# potential code:
#from snorkel.models import Candidate, candidate_subclass
#VirusHost = candidate_subclass('VirusHost', ['virus', 'host'])

### Writing a basic `CandidateExtractor`

Next, we'll write a basic function to extract **candidate spouse relation mentions** from the corpus.  The [Spacy](https://spacy.io/) parser we used performs _named entity recognition_ for us.

We will extract `Candidate` objects of the `Spouse` type by identifying, for each `Sentence`, all pairs of n-grams (up to 7-grams) that were tagged as people. (An n-gram is a span of text made up of n tokens.) We do this with three objects:

* A `ContextSpace` defines the "space" of all candidates we even potentially consider; in this case we use the `Ngrams` subclass, and look for all n-grams up to 7 words long

* A `Matcher` heuristically filters the candidates we use.  In this case, we just use a pre-defined matcher which looks for all n-grams tagged by Spacy as "PERSON". The keyword argument `longest_match_only` means that we'll skip n-grams contained in other n-grams.

* A `CandidateExtractor` combines this all together!